## Imports

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from IPython.display import Image
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from shutil import copy2
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

## Lendo o Dataset

In [2]:
pokemon_df = pd.read_csv('pokemon.csv')
pokemon_df

,Name,Type1,Type2
0,bulbasaur,Grass,Poison
1,ivysaur,Grass,Poison
2,venusaur,Grass,Poison
3,charmander,Fire,NaN
4,charmeleon,Fire,NaN
...,...,...,...
804,stakataka,Rock,Steel
805,blacephalon,Fire,Ghost
806,zeraora,Electric,NaN
807,meltan,Steel,NaN


### Adicionando path da imagem na row correspondente

Algumas imagens possuem fundo transparente, então foi feita a conversão para RGBA

In [13]:
pokemon_imgs = os.listdir('images')

sorted_imgs = pokemon_imgs.copy()
for pokemon in pokemon_imgs:
    if pokemon.split('.')[1] == 'png':
        pk_img = Image.open(f'images/{pokemon}').convert("RGBA")
        pk_img.save(f'images/{pokemon}')
    pokemon_name = pokemon.split('.')[0]
    index = pokemon_df[pokemon_df.Name == pokemon_name].index.values[0]
    sorted_imgs[index] = pokemon

OSError: cannot write mode RGBA as JPEG

In [4]:
pokemon_df['Pokemon Image'] = sorted_imgs

### Separando imagens em Treino e Validação

Add Teste

In [5]:
if not os.path.exists('train/'):
    os.mkdir('train/')
if not os.path.exists('val/'):
    os.mkdir('val/')

In [6]:
for pk_type in pokemon_df.Type1.unique():
    if not os.path.exists(f'train/{pk_type}/'):
        os.mkdir('train/'+str(pk_type)+'/')
    if not os.path.exists(f'val/{pk_type}/'):
        os.mkdir('val/'+str(pk_type)+'/')

In [7]:
X_train, X_val, y_train, y_val = train_test_split(pokemon_df['Pokemon Image'], pokemon_df.Type1, 
                                                    test_size=0.3, stratify=pokemon_df.Type1, random_state=42)

In [8]:
for from_path, to_path in zip(X_train.values, y_train.values):
    if not os.path.exists( f'train/{to_path}'):
        copy2(f'images/{from_path}', f'train/{to_path}')
    

In [9]:
for from_path, to_path in zip(X_val.values, y_val.values):
    if not os.path.exists( f'val/{to_path}'):
        copy2(f'images/{from_path}', f'val/{to_path}')

In [10]:
train_datagen = ImageDataGenerator().flow_from_directory('train/',
                                                         target_size = (64, 64),
                                                         batch_size = 32)

val_datagen = ImageDataGenerator().flow_from_directory('val/',
                                                         target_size = (64, 64),
                                                         batch_size = 32)

Found 566 images belonging to 18 classes.
Found 243 images belonging to 18 classes.


## Testando CNNs

In [11]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64,64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))
classifier.add(Flatten())
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dense(units = 18, activation = 'sigmoid'))

classifier.compile(loss='categorical_crossentropy', optimizer= "adam", metrics=['accuracy'])

In [12]:
classifier.fit(train_datagen,
               epochs = 20,
               validation_data = val_datagen)

C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/20
18/18 [==============================] - 7s 392ms/step - loss: 22.4790 - accuracy: 0.1079 - val_loss: 3.6039 - val_accuracy: 0.0947
Epoch 2/20
18/18 [==============================] - 1s 58ms/step - loss: 2.9684 - accuracy: 0.1605 - val_loss: 2.7547 - val_accuracy: 0.1358
Epoch 3/20
18/18 [==============================] - 1s 58ms/step - loss: 2.5243 - accuracy: 0.2146 - val_loss: 2.6962 - val_accuracy: 0.1893
Epoch 4/20
18/18 [==============================] - 1s 58ms/step - loss: 2.2076 - accuracy: 0.3096 - val_loss: 2.7025 - val_accuracy: 0.1770
Epoch 5/20
18/18 [==============================] - 1s 59ms/step - loss: 1.9468 - accuracy: 0.4241 - val_loss: 2.7241 - val_accuracy: 0.1893
Epoch 6/20
18/18 [==============================] - 1s 58ms/step - loss: 1.6166 - accuracy: 0.5231 - val_loss: 2.7791 - val_accuracy: 0.2058
Epoch 7/20
18/18 [==============================] - 1s 59ms/step - loss: 1.2236 - accuracy: 0.6596 - val_loss: 2.8909 - val_accuracy: 0.2058
Epoch 8/20
